In [6]:
import requests
import json
import sseclient
import time
from datetime import datetime
import pandas as pd
import os
from collections import defaultdict

✓ Librairies importées avec succès
Date de démarrage : 2025-12-13 11:25:03



In [9]:
ENTITIES = {
    "bob_vs_society": {
        "name": "Bob vs. Society",
        "type": "film",
        "wiki_title": "Bob_vs._Society"
    },
    "fred_astaire": {
        "name": "Fred Astaire",
        "type": "actor",
        "wiki_title": "Fred_Astaire"
    },
    "shawshank_redemption": {
        "name": "The Shawshank Redemption",
        "type": "film",
        "wiki_title": "The_Shawshank_Redemption"
    },
    "christopher_nolan": {
        "name": "Christopher Nolan",
        "type": "director",
        "wiki_title": "Christopher_Nolan"
    },
    "the_godfather": {
        "name": "The Godfather",
        "type": "film",
        "wiki_title": "The_Godfather"
    }
}

print("Entities :")
for key, entity in ENTITIES.items():
    print(f"  • {entity['name']} ({entity['type']})")
print()

Entities :
  • Bob vs. Society (film)
  • Fred Astaire (actor)
  • The Shawshank Redemption (film)
  • Christopher Nolan (director)
  • The Godfather (film)



In [11]:
# ===========================
# 3. CONFIGURATION DES FICHIERS DE SORTIE
# ===========================

# Créer un dossier pour stocker les données
OUTPUT_DIR = "wikimedia_stream_data"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Chemins des fichiers
METRICS_FILE = os.path.join(OUTPUT_DIR, "metrics.csv")
ALERTS_FILE = os.path.join(OUTPUT_DIR, "alerts.csv")
SUMMARY_FILE = os.path.join(OUTPUT_DIR, "summary.csv")

print(f"Metrics : {METRICS_FILE}")
print(f"Alerts : {ALERTS_FILE}")
print(f"Summary : {SUMMARY_FILE}\n")

Metrics : wikimedia_stream_data\metrics.csv
Alerts : wikimedia_stream_data\alerts.csv
Summary : wikimedia_stream_data\summary.csv



In [13]:
metrics_data = []
alerts_data = []

summary_stats = defaultdict(lambda: {
    "total_edits": 0,
    "total_bytes_changed": 0,
    "unique_users": set(),
    "anonymous_edits": 0,
    "major_changes": 0
})


In [15]:

def is_entity_page(title):
    """Vérifie si la page modifiée correspond à une de nos entités"""
    for entity_key, entity_info in ENTITIES.items():
        if entity_info['wiki_title'] == title:
            return entity_key, entity_info['name']
    return None, None

def process_change_event(change):
    """
    Traite un événement de modification et extrait les métriques
    
    Paramètres:
    -----------
    change : dict
        Événement de modification reçu du stream Wikimedia
        
    Retourne:
    ---------
    dict ou None : Métriques extraites ou None si pas pertinent
    """
    try:
        # Vérifier que c'est un événement de type "edit"
        if change.get('type') != 'edit':
            return None
        
        # Vérifier que c'est une page anglaise
        if change.get('wiki') != 'enwiki':
            return None
        
        title = change.get('title', '').replace(' ', '_')
        entity_key, entity_name = is_entity_page(title)
        
        if entity_key is None:
            return None
        
        # Extraire les informations importantes
        timestamp = datetime.fromtimestamp(change.get('timestamp', 0))
        user = change.get('user', 'Unknown')
        is_bot = change.get('bot', False)
        is_anonymous = 'user' in change and not change.get('user')
        bytes_changed = change.get('length', {}).get('new', 0) - change.get('length', {}).get('old', 0)
        comment = change.get('comment', '')[:100]  # Limiter à 100 caractères
        
        metric = {
            'timestamp': timestamp,
            'entity_key': entity_key,
            'entity_name': entity_name,
            'user': user,
            'is_bot': is_bot,
            'is_anonymous': is_anonymous,
            'bytes_changed': bytes_changed,
            'comment': comment
        }
        
        return metric
        
    except Exception as e:
        print(f"Erreur lors du traitement : {e}")
        return None

def check_for_alerts(metric):
    """
    Vérifie si une métrique déclenche une alerte
    
    Critères d'alerte:
    - Modification > 1000 bytes (changement majeur)
    - Modification par utilisateur anonyme
    """
    alerts = []
    
    # Alerte 1: Changement majeur
    if abs(metric['bytes_changed']) > 1000:
        alerts.append({
            'timestamp': metric['timestamp'],
            'entity_name': metric['entity_name'],
            'alert_type': 'MAJOR_CHANGE',
            'description': f"Changement de {metric['bytes_changed']} bytes",
            'user': metric['user']
        })
    
    # Alerte 2: Utilisateur anonyme
    if metric['is_anonymous']:
        alerts.append({
            'timestamp': metric['timestamp'],
            'entity_name': metric['entity_name'],
            'alert_type': 'ANONYMOUS_EDIT',
            'description': "Modification par utilisateur anonyme",
            'user': metric['user']
        })
    
    return alerts

def update_summary_stats(metric):
    """Met à jour les statistiques résumées"""
    key = metric['entity_key']
    summary_stats[key]['total_edits'] += 1
    summary_stats[key]['total_bytes_changed'] += metric['bytes_changed']
    summary_stats[key]['unique_users'].add(metric['user'])
    
    if metric['is_anonymous']:
        summary_stats[key]['anonymous_edits'] += 1
    
    if abs(metric['bytes_changed']) > 1000:
        summary_stats[key]['major_changes'] += 1

def save_data():
    """Sauvegarde les données dans les fichiers CSV"""
    # Sauvegarder les métriques
    if metrics_data:
        df_metrics = pd.DataFrame(metrics_data)
        df_metrics.to_csv(METRICS_FILE, index=False)
        print(f"✓ {len(metrics_data)} métriques sauvegardées")
    
    # Sauvegarder les alertes
    if alerts_data:
        df_alerts = pd.DataFrame(alerts_data)
        df_alerts.to_csv(ALERTS_FILE, index=False)
        print(f"✓ {len(alerts_data)} alertes sauvegardées")
    
    # Sauvegarder le résumé
    summary_list = []
    for entity_key, stats in summary_stats.items():
        summary_list.append({
            'entity_key': entity_key,
            'entity_name': ENTITIES[entity_key]['name'],
            'total_edits': stats['total_edits'],
            'total_bytes_changed': stats['total_bytes_changed'],
            'unique_users': len(stats['unique_users']),
            'anonymous_edits': stats['anonymous_edits'],
            'major_changes': stats['major_changes']
        })
    
    if summary_list:
        df_summary = pd.DataFrame(summary_list)
        df_summary.to_csv(SUMMARY_FILE, index=False)
        print(f"✓ Résumé sauvegardé pour {len(summary_list)} entités")

print("Fonctions utilitaires définies ✓\n")

Fonctions utilitaires définies ✓



In [26]:
def connect_to_stream(duration_seconds=300):
    """
    Se connecte au stream Wikimedia EventStreams avec gestion des déconnexions
    """
    url = 'https://stream.wikimedia.org/v2/stream/recentchange'
    
    headers = {
        'User-Agent': 'IMDb-Stream-Project/1.0 (Educational; Contact: votre.email@example.com)',
        'Accept': 'text/event-stream'
    }
    
    print("="*60)
    print("DÉMARRAGE DE LA COLLECTE")
    print("="*60)
    print(f"URL : {url}")
    print(f"Durée : {duration_seconds} secondes ({duration_seconds/60:.1f} minutes)")
    print(f"Heure de début : {datetime.now().strftime('%H:%M:%S')}")
    print("="*60 + "\n")
    
    start_time = time.time()
    event_count = 0
    reconnect_count = 0
    max_reconnects = 5
    
    while time.time() - start_time < duration_seconds and reconnect_count < max_reconnects:
        try:
            if reconnect_count > 0:
                print(f"\n🔄 Reconnexion ({reconnect_count}/{max_reconnects})...")
                time.sleep(2)  # Attendre 2 secondes avant de reconnecter
            else:
                print("🔌 Connexion au stream EventStreams...")
            
            response = requests.get(
                url, 
                stream=True, 
                headers=headers,
                timeout=60  # Timeout de 60s pour détecter les problèmes
            )
            
            if response.status_code != 200:
                print(f"❌ Erreur HTTP {response.status_code}")
                reconnect_count += 1
                continue
            
            if reconnect_count == 0:
                print("✓ Connexion établie au stream EventStreams")
                print("⏳ En attente d'événements...\n")
            else:
                print("✓ Reconnecté !")
            
            buffer = ""
            
            for chunk in response.iter_content(chunk_size=1024, decode_unicode=True):
                # Vérifier le temps écoulé
                elapsed = time.time() - start_time
                if elapsed > duration_seconds:
                    print(f"\n⏰ Durée atteinte ({elapsed:.1f}s)")
                    return  # Sortir complètement
                
                if not chunk:
                    continue
                
                buffer += chunk
                
                # Parser les événements
                while '\n\n' in buffer:
                    event_text, buffer = buffer.split('\n\n', 1)
                    
                    if not event_text.strip():
                        continue
                    
                    # Extraire la ligne data:
                    data_line = None
                    for line in event_text.split('\n'):
                        if line.startswith('data: '):
                            data_line = line[6:]
                            break
                    
                    if not data_line:
                        continue
                    
                    event_count += 1
                    
                    if event_count % 100 == 0:
                        print(".", end="", flush=True)
                    
                    if event_count == 1:
                        print(f"✓ Premier événement reçu après {elapsed:.1f}s\n")
                    
                    try:
                        change = json.loads(data_line)
                        
                        if event_count % 500 == 0:
                            print(f"\n📊 {event_count} événements en {elapsed:.1f}s")
                            if 'wiki' in change:
                                print(f"   Exemple : {change.get('wiki')} - {change.get('title', 'N/A')}")
                        
                        metric = process_change_event(change)
                        
                        if metric:
                            print(f"\n\n🎯 ÉVÉNEMENT DÉTECTÉ !")
                            print(f"   Entité : {metric['entity_name']}")
                            print(f"   User : {metric['user']}")
                            print(f"   Changement : {metric['bytes_changed']:+d} bytes\n")
                            
                            metrics_data.append(metric)
                            
                            alerts = check_for_alerts(metric)
                            if alerts:
                                print(f"   ⚠️  {len(alerts)} ALERTE(S) !")
                                for alert in alerts:
                                    print(f"      → {alert['alert_type']}")
                                alerts_data.extend(alerts)
                            
                            update_summary_stats(metric)
                            
                    except json.JSONDecodeError:
                        continue
                    except Exception:
                        continue
            
            # Si on arrive ici, la boucle s'est terminée normalement
            break
            
        except (requests.exceptions.ChunkedEncodingError, 
                requests.exceptions.ConnectionError,
                requests.exceptions.Timeout) as e:
            reconnect_count += 1
            if reconnect_count < max_reconnects:
                print(f"\n⚠️  Déconnexion détectée (tentative {reconnect_count}/{max_reconnects})")
            else:
                print(f"\n❌ Trop de déconnexions ({max_reconnects}). Arrêt.")
                
        except KeyboardInterrupt:
            print("\n⏸️  Interrompu par l'utilisateur")
            break
            
        except Exception as e:
            print(f"\n❌ Erreur inattendue : {type(e).__name__}")
            reconnect_count += 1
            if reconnect_count >= max_reconnects:
                break
    
    # Sauvegarder à la fin
    print("\n" + "="*60)
    print("SAUVEGARDE")
    print("="*60)
    save_data()
    
    print("\n" + "="*60)
    print("STATISTIQUES FINALES")
    print("="*60)
    print(f"Événements traités : {event_count}")
    print(f"Événements pertinents : {len(metrics_data)}")
    print(f"Alertes : {len(alerts_data)}")
    print(f"Durée : {time.time() - start_time:.1f}s")
    print(f"Reconnexions : {reconnect_count}")
    print("="*60 + "\n")

In [27]:
# ===========================
# 7. EXÉCUTION DE LA COLLECTE
# ===========================

# NOTE : Décommenter la ligne suivante pour lancer la collecte
connect_to_stream(duration_seconds=300)  # Collecter pendant 5 minutes

print("""
INSTRUCTIONS POUR EXÉCUTER LA COLLECTE
======================================

1. Installer la librairie sseclient-py :
   pip install sseclient-py

2. Décommenter la ligne :
   connect_to_stream(duration_seconds=300)

3. Exécuter cette cellule

4. Attendre la fin de la collecte (5 minutes par défaut)

5. Les données seront sauvegardées dans le dossier 'wikimedia_stream_data/'

AJUSTER LA DURÉE
================
Pour collecter pendant plus ou moins longtemps, modifier le paramètre :
- 60 secondes = 1 minute
- 300 secondes = 5 minutes
- 3600 secondes = 1 heure
""")


DÉMARRAGE DE LA COLLECTE
URL : https://stream.wikimedia.org/v2/stream/recentchange
Durée : 300 secondes (5.0 minutes)
Heure de début : 11:49:58

🔌 Connexion au stream EventStreams...
✓ Connexion établie au stream EventStreams
⏳ En attente d'événements...

✓ Premier événement reçu après 2.0s

.....
📊 500 événements en 15.1s
   Exemple : commonswiki - Category:Files uploaded with OpenRefine by PantheraLeo1359531
.....
📊 1000 événements en 28.4s
   Exemple : commonswiki - Category:Files uploaded with OpenRefine by PantheraLeo1359531
.....
📊 1500 événements en 41.6s
   Exemple : wikidatawiki - Q73942392
.....
📊 2000 événements en 54.5s
   Exemple : commonswiki - File:STP 20160201 955 (39555198894).jpg
.....
📊 2500 événements en 69.1s
   Exemple : commonswiki - Category:Digitales Oberflächenmodell 1m (IT.NRW)
.....
📊 3000 événements en 81.5s
   Exemple : arwiktionary - unis
.....
📊 3500 événements en 95.4s
   Exemple : commonswiki - Category:Images from NOAA uploaded by PantheraLeo1359531
.

In [29]:

# ===========================
# 8. ANALYSE DES RÉSULTATS
# ===========================

def analyze_results():
    """Analyse et affiche les résultats collectés"""
    print("\n" + "="*60)
    print("ANALYSE DES RÉSULTATS")
    print("="*60 + "\n")
    
    # Charger les données
    try:
        df_metrics = pd.read_csv(METRICS_FILE)
        df_alerts = pd.read_csv(ALERTS_FILE)
        df_summary = pd.read_csv(SUMMARY_FILE)
        
        print("📊 RÉSUMÉ PAR ENTITÉ")
        print("-" * 60)
        print(df_summary.to_string(index=False))
        
        print("\n\n⚠️  ALERTES RÉCENTES (5 dernières)")
        print("-" * 60)
        if len(df_alerts) > 0:
            print(df_alerts.tail().to_string(index=False))
        else:
            print("Aucune alerte enregistrée")
        
        print("\n\n📝 MODIFICATIONS RÉCENTES (5 dernières)")
        print("-" * 60)
        if len(df_metrics) > 0:
            print(df_metrics[['timestamp', 'entity_name', 'user', 'bytes_changed']].tail().to_string(index=False))
        else:
            print("Aucune modification enregistrée")
        
        print("\n" + "="*60 + "\n")
        
    except FileNotFoundError:
        print("⚠️  Aucune donnée trouvée. Exécutez d'abord la collecte !")

# Décommenter pour analyser les résultats après la collecte
analyze_results()


ANALYSE DES RÉSULTATS

⚠️  Aucune donnée trouvée. Exécutez d'abord la collecte !
